### Plugin Controller API and Developer Guide

** This interactive guide leverages an Alpine Linux container running Jupyter Notebook.  Its purpose is to support the Plugin developer in building a PCP compliant Plugin.  Aside from guiding the Plugin developer, this Plugin template (using Jupyter Notebook) also serves as an interactive presentation of the PCP Plugin API. **

** First, re-save this Plugin template (this Jupyter Notebook) as [pluginName].ipynb, by selecting FILE-->RENAME from the top menu above. You will also want to save frequently, with FILE-- >SAVE AS CHECKPOINT, as well as using the DOWNLOAD AS .pynb, which will allow you to archive your plugin development progress in the event your system is restarted. While it is also possible to map your host machine folder to the container folder, allowing access to your Plugin notebook file, it is preferable to deliberately save and track the progress of your work using features within this Jupyter Notebook application. **

** The following steps will guide the plugin developer through the PCP API and required steps to build a PCP compliant plugin. It does this using the interactive nature of Jupyter notebook, as well as a background helper script (python-builder.py) which converts the [pluginName].ipynb to an importable Python module which will complete creation of the plugin and add it to the list of plugins found by the PCP application. **


### Introduction to using Jupyter Notebook

** In case you have not used Jupyter Notebook before, we will start with a quick overview of the tool. This tool provides an interactive execution environment in a number of different scripting and programming languages, including: **

* bash, C, C++, Go, Javascript (node.js), Octave, pig, Python 2, Python 3, R, Ruby, Spark - PySpark, Spark - SQL, Spark - Scala, and Spark - SparkR

** This Notebook is able to use Python 2, Python 3, bash, and Javascript (node.js). The docker container running this Notebook is derived from nbgallery/jupyter-alpine, which keeps image size low, installing language support on first use of the KERNEL-->CHANGE KERNEL selection from the top menu. As a result, if additional language support is required (KERNEL showing on the right top corner of this Notebook), the running container must have access to internet-accessible repositories, or we would rebuild this custom image after loading the desired Kernel. So for anyone using this container, language support for the cells that are executing code are limited to:

 * bash, Python 2, and Python 3
 
 ** This custom Jupyter Notebook is pre-installed with support for Python 2, Python 3 and bash. Additional language support is straightforward (it will autoinstall on first use), but requires a container re-build with access to the internet-connected repositories. The language currently being "interpreted" is displayed in the upper right of this notebook. As an illustration of the capabilities of the Notebook, change this language by selecting "bash" under the Kernel-->Change Kernel--> bash. **
 
** Next, click in the cell below that contains the bash script. Once in that cell, select CELL-- >RUN CELLS. The result of the script should be displayed below the cell being executed. **



In [ ]:
greeting="hello, world"                   `# Regular variable assignment ` 
declare -n message=greeting               `# Declare the nameref`
echo $message                             `# Access through it`
message="bye now"                         `# Assign through the nameref`
echo $greeting                            `# Demonstrate the change`

** The following code installs docker to enable interaction with the host with respect to docker commands. The second block, using the Python 3 kernel, brings down the "BRAIN" functionality that will be needed in order to start the Plugin development process. DO NOT execute the code in the two cells below. This will already have been completed prior to delivery of the docker container backing this notebook (ramrodpcp/devguide-api). Also, if you do not have internet connection / access to the apk (alpine) code repositories and pypi (the python repository), the code execution will fail.  **

In [ ]:
apk update                               `# updates Alpine linux package list`
apk add docker                           `# Adds docker to the container that is running this Jupyter Notebook`

In [ ]:
pip install ramrodbrain                  `# Adds the python module for rethink interaction, currently released on pypi`

** You should have started this notebook from a host that has Docker version 18ce installed, and with the executable docker file at /run/docker.sock on that host. This Jupyter notebook application will have been started with the following command: **

* docker run -it -v /run/docker.sock:/run/docker.sock ramrodpcp/devguide-api:dev

** replacing `<host-user-folder>` with the desired host folder. **

** This will start the docker container running this PCP Plugin Controller API and Developer Guide, using the docker installation on your host machine to expose the Jupyter Notebook application via https (requires an SSL configuration, otherwise falling back to insecure http) on port 443. **

** Now go back to the menu, and change back to Python 3, by selecting KERNEL-->CHANGE KERNEL-->Python 3. Make sure the notebook is running Python 3 by looking in the upper right of the notebook. You should see "Python 3" displayed there. Your plugin controller will be written in Python 3 in order to ensure compatibility with the PCP controller application. You will have an opportunity to test your code changes within this Python notebook, as well as performing an integration test after completion of your plugin development. **

** Note that you can also use this notebook to develop and test your controller - client interaction, using whatever scripting or programming languages and techniques required. If this notebook supports the language (supported languages in the list highlighted in the intro section above), you can test that interaction by adding a cell from above using INSERT-->CELL BELOW from the top menu. **

** In the next section, this guide will walk you through the steps to build, verify, test, and subsequently deploy your custom plugin controller to be used with the PCP application. **

### PCP API Requirements for the Plugin Developer

** In the following sections, we will import the "BRAIN" functionality into this notebook in order to build the command set that you will need your plugin to support, saving the results as a JSON file. Next, we will walk through the necessary functions in the Plugin Controller itself, upload the JSON command set that you previously built, and load the resulting module into a test container to ensure it is operating as expected prior to performing integration tests against the PCP application. Next, we will deploy your newly created Plugin with the PCP Application and checkout its operation within the running application suite. Finally, we will tear down the PCP application suite and shut down this helper notebook and its underlying application. **

### Setting up and Verifying the Plugin Controller Metadata (Converting commmand data to protocol buffers)

** We will start by building the names and executable commands that your controller will need to communicate with it's client. **

** We first load the module that will perform verification tests against the metadate you will be using, in order to ensure the resulting command protocol object is compatible with the PCP application. ** 

** Execute the code block below, by clicking in the cell block below, then selecting CELL-->RUN CELLS at the top menu. **

In [1]:
import brain

** Plugin commands will be defined in Python 3, so make sure the running Kernel is Python 3 as shown in the upper right corner of this notebook. The Plugin commands will be saved as a Python list of dictionaries, with each dictionary containing the required information for each command, illustrated in the following example:

`GoodCommand = {"CommandName" : "anystring",
              "Tooltip" : "otherstring",      
              "Output" : True | False,                  # Select True for required Output, False for no output
              "Inputs" : [{"Name" : "string",           # Name, Type, Tooltip, Value are minimum input requirements
                          "Type" : "String",            # for each command.
                          "Tooltip : "String",
                          "Value" : "String"}],
              "OptionalInputs" : [{"Name" : "string",   # The same Name, Type, Tooltip, and Value are required for
                                   "Type" : "String",   # each optional input, if any.
                                   "Tooltip" : "String",
                                   "Value" : "String"}]}`
                                   
** The data below repeats the above in an executable cell. Run the cell below. The output should be "True", which indicates that the example command is valid, and will load properly into the PCP application. **

In [11]:
GoodCommand = {"CommandName" : "anystring",
              "Tooltip" : "otherstring",      
              "Output" : True | False,                  # Select True for required Output, False for no output
              "Inputs" : [{"Name" : "string",           # Name, Type, Tooltip, Value are minimum input requirements
                          "Type" : "str",               # for each command.
                          "Tooltip" : "string",
                          "Value" : "String"}],
              "OptionalInputs" : [{"Name" : "string",   # The same Name, Type, Tooltip, and Value are required for
                                   "Type" : "str",   # each optional input, if any.
                                   "Tooltip" : "string",
                                   "Value" : "string"}]}

brain.checks.verify(GoodCommand,brain.brain_pb2.Command())

True

** Taking a simple, but practical example, let uss implement the commands required to grab web page data from a running server using wget. We will need to use the command wget, and a placeholder for the webpage to be retrieved. After deployment, the PCP application will allow us to edit the command sequence if needed, such as the web address, so we will keep it generic. Run the cell below, and look again for a "True" output, indicating the command is valid as built **

In [13]:
cmdWget = {"CommandName" : "wget",
          "Tooltip" : "Retrieves raw webpage files from targeted systems",
          "Output" : True,
          "Inputs" : [{"Name" : "Target IP Address",
                     "Type" : "url",
                     "Tooltip" : "Modify the URL to the desired target IP address",
                     "Value" : "http://192.168.1.100"}]}

brain.checks.verify(cmdWget , brain.brain_pb2.Command())

True

### Setting up the Plugin Controller

** Next, we will walk through the controller plugin code, setting up the commands that your Plugin Controller will need to use in order to interact with the PCP application that will enable commiunication with the malware/client that it is intended to control. **

In [2]:
cd /home/linpy/backend-interpreter/plugin_interpreter/src
cat controller_plugin.py

"""Plugin Template Module
TODO:
- add helper function for function advertisement
- add helper function for job request
"""

from abc import ABC, abstractmethod
from os import environ
from queue import Empty

from src import rethink_interface


class ControllerPlugin(ABC):
    """
    The ControllerPlugin class is an Abstract Base Class from which plugin
    subclasses can be derived. Plugins should inherit this base class and
    implement any subset of its methods that is required.
    #
    For proper instantiation, plugin subclasses should be initialized
    with a 'name' string, and 'functionality' dictionary which describes
    the functions available in the plugin.
    #
    Port allocation is done automatically by the controller, and upon
    instantiation the plugin will be given a PORT environment variable
    where it should be running its server.
    #
    The initialize_queues method *SHOUD NOT* be overridden by the
    inheriting class, as the Supervisor will attempt to in

            }
        """
        try:
            job = self.db_recv.get_nowait()
        except Empty:
            job = None

        if job:
            self._update_job(job["id"])
        return job

    def _respond_output(self, job, output):
        """Provide job response output

        This method is a helper method for the plugin
        which is inheriting this base class. The plugin
        must pass this function the job object it
        received from the _request_job helper function
        and the corresponding output from the
        command.

        This method also performs some basic type
        checking on the output.

        Arguments:
            job {dict} -- the dictionary object for
            the job received from the database/frontend.
            output {} -- [description]
        """
        if not isinstance(output, (bytes, str, int, float)):
            raise TypeError
        self.DBI.send_output({
            "job": job,
            "output": outp

** Now open SamplePlugin.ipynb to learn how to buid a custom plugin **